In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium.plugins import MarkerCluster
from IPython.display import HTML
from folium.plugins import MarkerCluster
import ipywidgets as widgets
from IPython.display import display
from geopy.distance import geodesic
from pytz import timezone


## Load crime data & slice for the school year 2022-2023

In [4]:
df_crimes_2023 = pd.read_csv('../data/df_crimes_2023.csv')
df_crimes_2023.head()
df_crimes_2023.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30336 entries, 0 to 30335
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   INCIDENT_NUMBER      30336 non-null  object 
 1   OFFENSE_CODE         30336 non-null  int64  
 2   OFFENSE_CODE_GROUP   0 non-null      float64
 3   OFFENSE_DESCRIPTION  30336 non-null  object 
 4   DISTRICT             30243 non-null  object 
 5   REPORTING_AREA       30336 non-null  object 
 6   SHOOTING             30336 non-null  int64  
 7   OCCURRED_ON_DATE     30336 non-null  object 
 8   YEAR                 30336 non-null  int64  
 9   MONTH                30336 non-null  int64  
 10  DAY_OF_WEEK          30336 non-null  object 
 11  HOUR                 30336 non-null  int64  
 12  UCR_PART             0 non-null      float64
 13  STREET               30336 non-null  object 
 14  Lat                  26896 non-null  float64
 15  Long                 26896 non-null 

In [6]:
df_crimes2022 = pd.read_csv('../data/Crimes_report_2022.csv')
df_crimes2022.head()
df_crimes2022.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73852 entries, 0 to 73851
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   INCIDENT_NUMBER      73852 non-null  object 
 1   OFFENSE_CODE         73852 non-null  int64  
 2   OFFENSE_CODE_GROUP   0 non-null      float64
 3   OFFENSE_DESCRIPTION  73852 non-null  object 
 4   DISTRICT             73681 non-null  object 
 5   REPORTING_AREA       73852 non-null  object 
 6   SHOOTING             73852 non-null  int64  
 7   OCCURRED_ON_DATE     73852 non-null  object 
 8   YEAR                 73852 non-null  int64  
 9   MONTH                73852 non-null  int64  
 10  DAY_OF_WEEK          73852 non-null  object 
 11  HOUR                 73852 non-null  int64  
 12  UCR_PART             0 non-null      float64
 13  STREET               73851 non-null  object 
 14  Lat                  70044 non-null  float64
 15  Long                 70044 non-null 

/var/folders/gq/9n8m2mf944d9l259z26w1bb40000gn/T/ipykernel_47563/3794551377.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df_crimes2022 = pd.read_csv('../data/Crimes_report_2022.csv')


In [7]:
crimes_2022_2023_merged = pd.concat([df_crimes2022,df_crimes_2023])

In [8]:
crimes_2022_2023_merged.drop(['OFFENSE_CODE_GROUP','OFFENSE_CODE','UCR_PART'],axis = 'columns',inplace = True)

In [10]:
crimes_2022_2023_merged['OCCURRED_ON_DATE'] = pd.to_datetime(
    crimes_2022_2023_merged['OCCURRED_ON_DATE'],
    utc=True,
    format='mixed'
).dt.tz_convert('America/New_York')


In [11]:
crimes_2022_2023_filtered = crimes_2022_2023_merged[crimes_2022_2023_merged['OCCURRED_ON_DATE'] >= pd.to_datetime('2022-09-08 00:00:00', utc=True).tz_convert('America/New_York')]

## Keep only crimes that involved shooting to create the map for 2022-2023

In [12]:
crimes_2022_2023_filtered = crimes_2022_2023_filtered[crimes_2022_2023_filtered['SHOOTING'] == 1]

In [13]:
crimes_2022_2023_filtered.head()

,INCIDENT_NUMBER,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,STREET,Lat,Long,Location
50986,222068976,INVESTIGATE PERSON,D14,,1,2022-09-08 09:04:00-04:00,2022,9,Thursday,13,COMMONWEALTH AVE,42.349066,-71.134147,"(42.34906577769542, -71.13414695373906)"
51068,222069135,INVESTIGATE PROPERTY,C11,250,1,2022-09-08 16:54:00-04:00,2022,9,Thursday,20,SAVIN HILL AVE,42.312197,-71.052048,"(42.312197463106756, -71.05204832519735)"
51151,222069199,BALLISTICS EVIDENCE/FOUND,A1,,1,2022-09-09 02:34:00-04:00,2022,9,Friday,6,NEW CHARDON ST,42.362607,-71.061327,"(42.3626074984039, -71.06132725274803)"
51301,222069437,BALLISTICS EVIDENCE/FOUND,B2,,1,2022-09-09 17:24:00-04:00,2022,9,Friday,21,DUDLEY ST,42.321970,-71.072425,"(42.32196970405995, -71.07242528935052)"
51524,222069744,INVESTIGATE PROPERTY,B3,,1,2022-09-10 19:05:00-04:00,2022,9,Saturday,23,RIPLEY RD,42.299263,-71.076265,"(42.2992630326008, -71.0762651335127)"


In [14]:
crimes_2022_2023_filtered.dropna(subset=['Location'], inplace=True)

In [15]:
crimes_2022_2023_filtered['Location'].isna().sum()

0

## Do the same for school year 2021-2022

In [17]:
df_crimes2021 = pd.read_csv('../data/Crimes_report_2021.csv')
df_crimes2021.head()

/var/folders/gq/9n8m2mf944d9l259z26w1bb40000gn/T/ipykernel_47563/2354670698.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df_crimes2021 = pd.read_csv('../data/Crimes_report_2021.csv')


,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,Lat,Long,Location
0,212021001,619,NaN,LARCENY ALL OTHERS,B2,181,0,2021-01-01 00:00:00,2021,1,Friday,0,NaN,ALLERTON ST & MAGAZINE ST\nROXBURY MA 02119\n...,42.327551,-71.069162,"(42.327550999587764, -71.0691619893362)"
1,212025963,3201,NaN,PROPERTY - LOST/ MISSING,B3,465,0,2021-01-01 00:00:00,2021,1,Friday,0,NaN,WHITFIELD ST,42.284826,-71.091374,"(42.28482576580488, -71.09137368938802)"
2,212022379,1102,NaN,FRAUD - FALSE PRETENSE / SCHEME,C6,574,0,2021-01-01 00:00:00,2021,1,Friday,0,NaN,MARINE RD,42.309719,-71.104294,"(42.30971856767274, -71.10429431787648)"
3,212019050,615,NaN,LARCENY THEFT OF MV PARTS & ACCESSORIES,A1,77,0,2021-01-01 00:00:00,2021,1,Friday,0,NaN,STANIFORD ST,42.361839,-71.059765,"(42.361838566564714, -71.05976489094158)"
4,212019422,3115,NaN,INVESTIGATE PERSON,B3,355,0,2021-01-01 00:00:00,2021,1,Friday,0,NaN,FULLER ST,42.297555,-71.059709,"(42.29755532959655, -71.05970910242573)"


In [18]:
df_crimes2021_2022_merged = pd.concat([df_crimes2021,df_crimes2022])

In [19]:
df_crimes2021_2022_merged.drop(['OFFENSE_CODE_GROUP','OFFENSE_CODE','UCR_PART'],axis = 'columns',inplace = True)

In [20]:
df_crimes2021_2022_merged['OCCURRED_ON_DATE'] = pd.to_datetime(df_crimes2021_2022_merged['OCCURRED_ON_DATE'], utc=True).dt.tz_convert('America/New_York')

In [21]:
crimes_2021_2022_filtered = df_crimes2021_2022_merged[(df_crimes2021_2022_merged['OCCURRED_ON_DATE'] >= pd.to_datetime('2021-09-09 00:00:00', utc=True).tz_convert('America/New_York')) &
                                                      (df_crimes2021_2022_merged['OCCURRED_ON_DATE'] <= pd.to_datetime('2022-06-22 00:00:00', utc=True).tz_convert('America/New_York'))]

## Keep only crimes that involved shooting to create the map for 2022-2023

In [22]:
crimes_2021_2022_filtered = crimes_2021_2022_filtered[crimes_2021_2022_filtered['SHOOTING'] == 1]

In [23]:
crimes_2021_2022_filtered.dropna(subset=['Location'], inplace=True)

In [24]:
crimes_2021_2022_filtered['Location'].isna().sum()

0

## Load School location data

In [26]:
boston_schools = pd.read_excel('../data/BPS_geolocation.xlsx')
boston_schools

,name,adress,latitude,longitude
0,Young Achievers Science & Math K-8,"20 Outlook Rd, Mattapan, MA, 02126",42.282320,-71.095000
1,"Winthrop, John Elementary","35 Brookford St, Dorchester, MA, 02125",42.318210,-71.075630
2,"Winship, F. Lyman Elementary","54 Dighton St, Brighton, MA, 02135",42.347610,-71.155350
3,West Zone Early Learning Center,"200 Heath St., Jamaica Plain, MA, 02130",42.325727,-71.106434
4,William E Carter School,"396 Northampton St Boston, MA 02118",42.326340,-71.106850
...,...,...,...,...
107,"Beethoven, Elementary School","5125 Washington St., West Roxbury, MA , 02132",42.263524,-71.155792
108,Baldwin Early Learning Pilot Academy,"121 Corey Rd., Brighton, MA , 02135",42.342060,-71.140472
109,Another Course to College,"612 Metropolitan Ave, Hyde Park, MA , 02136",42.265060,-71.117790
110,Alighier School,"37 Gove Street, East Boston, MA , 02128",42.371429,-71.037697


## Create the Map for 2022-2023

In [27]:
# Get shooting coordinates
shooting_latitude = crimes_2022_2023_filtered['Lat'].tolist()
shooting_longitude = crimes_2022_2023_filtered['Long'].tolist()
coords1 = list(zip(shooting_latitude, shooting_longitude))

# Create map
map = folium.Map(location=coords1[0], zoom_start=14, tiles="OpenStreetMap")

# Marker cluster for shooting locations
mc_shootings = MarkerCluster(name='Crimes')

# Create a list to store the crimes within the radius of schools
crimes_within_radius = []

# Create a dictionary to store the schools where crimes occurred within the radius
schools_within_radius = {}

# Add shooting markers to the map and check for crimes within the radius of schools
for coord, offense_description, occurred_date in zip(coords1, crimes_2022_2023_filtered['OFFENSE_DESCRIPTION'], crimes_2022_2023_filtered['OCCURRED_ON_DATE']):
    popup_description = f"<b>Description:</b> {offense_description}"
    popup_date = f"<b>Occurred On Date:</b> {occurred_date}"
    popup_content = f"{popup_description}<br>{popup_date}"
    marker = folium.Marker(location=coord, popup=popup_content,
                           icon=folium.Icon(color='red', icon='exclamation-circle', prefix='fa'))
    mc_shootings.add_child(marker)
    
    # Check for crimes within the radius of schools
    for index, row in boston_schools.iterrows():
        school_location = (row['latitude'], row['longitude'])
        radius = 200  # Radius in meters
        
        # Calculate the distance between the shooting location and the school location
        distance = geodesic(coord, school_location).meters
        
        # If the distance is within the radius, add the crime to the list and store the school name
        if distance <= radius:
            crimes_within_radius.append(offense_description)
            if offense_description not in schools_within_radius:
                schools_within_radius[offense_description] = [row['name']]
            else:
                schools_within_radius[offense_description].append(row['name'])

mc_shootings.add_to(map)

# Create a feature group for the school locations
school_locations = folium.FeatureGroup(name='School Locations')

# Style the school markers and circles
for index, row in boston_schools.iterrows():
    marker = folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=row['name'],
        icon=folium.Icon(color='blue', icon='home', prefix='fa')
    ).add_to(school_locations)
        
    folium.Circle(
        radius=200,
        location=[row['latitude'], row['longitude']],
        color="#3186cc",
        fill=True,
        fill_color="#3186cc",
        fill_opacity=0.4,
    ).add_to(school_locations)

# Add the school locations to the map
school_locations.add_to(map)

# Customize map appearance
folium.LayerControl().add_to(map)

# Create a label widget to display the number of crimes and crimes within the radius of schools
crimes_label = widgets.Label(value='Number of Crimes: 0')
crimes_within_radius_label = widgets.Label(value='Crimes within Radius: 0')
schools_within_radius_label = widgets.Label(value='Schools where Crimes Occurred within Radius:')

# Function to update the number of crimes and crimes within the radius based on the selected crime from the dropdown
def update_crimes_label(change):
    selected_crime = change['new']
    num_crimes = len(crimes_2022_2023_filtered[crimes_2022_2023_filtered['OFFENSE_DESCRIPTION'] == selected_crime])
    crimes_label.value = f'Number of Crimes: {num_crimes}'
    
    num_crimes_within_radius = crimes_within_radius.count(selected_crime)
    crimes_within_radius_label.value = f'Crimes within Radius: {num_crimes_within_radius}'
    
    if selected_crime in schools_within_radius:
        schools = schools_within_radius[selected_crime]
        schools_list = "\n".join(f"<li>{school.strip()}</li>" for school in schools)
        schools_within_radius_label.value = f'Schools where Crimes Occurred within Radius:<br><ul>{schools_list}</ul>'
    else:
        schools_within_radius_label.value = 'Schools where Crimes Occurred within Radius:'

# Create a dropdown widget with all different crimes
crime_dropdown = widgets.Dropdown(
    options=crimes_2022_2023_filtered['OFFENSE_DESCRIPTION'].unique(),
    description='Crime:'
)

# Create an HTML widget to display the schools within radius
schools_within_radius_label = widgets.HTML(value='Schools where Crimes Occurred within Radius:')

# Observe the change in the dropdown value and update the number of crimes and crimes within the radius
crime_dropdown.observe(update_crimes_label, 'value')

# Display the map and widgets
display(map)
display(widgets.VBox([crime_dropdown, crimes_label, crimes_within_radius_label, schools_within_radius_label]))


## Create the map for 2021-2022

In [ ]:
# Get shooting coordinates
shooting_latitude = crimes_2021_2022_filtered['Lat'].tolist()
shooting_longitude = crimes_2021_2022_filtered['Long'].tolist()
coords1 = list(zip(shooting_latitude, shooting_longitude))

# Create map
map = folium.Map(location=coords1[0], zoom_start=14, tiles="OpenStreetMap")

# Marker cluster for shooting locations
mc_shootings = MarkerCluster(name='Crimes')

# Create a list to store the crimes within the radius of schools
crimes_within_radius = []

# Create a dictionary to store the schools where crimes occurred within the radius
schools_within_radius = {}

# Add shooting markers to the map and check for crimes within the radius of schools
for coord, offense_description, occurred_date in zip(coords1, crimes_2021_2022_filtered['OFFENSE_DESCRIPTION'], crimes_2021_2022_filtered['OCCURRED_ON_DATE']):
    popup_description = f"<b>Description:</b> {offense_description}"
    popup_date = f"<b>Occurred On Date:</b> {occurred_date}"
    popup_content = f"{popup_description}<br>{popup_date}"
    marker = folium.Marker(location=coord, popup=popup_content,
                           icon=folium.Icon(color='red', icon='exclamation-circle', prefix='fa'))
    mc_shootings.add_child(marker)
    
    # Check for crimes within the radius of schools
    for index, row in boston_schools.iterrows():
        school_location = (row['latitude'], row['longitude'])
        radius = 200  # Radius in meters
        
        # Calculate the distance between the shooting location and the school location
        distance = geodesic(coord, school_location).meters
        
        # If the distance is within the radius, add the crime to the list and store the school name
        if distance <= radius:
            crimes_within_radius.append(offense_description)
            if offense_description not in schools_within_radius:
                schools_within_radius[offense_description] = [row['name']]
            else:
                schools_within_radius[offense_description].append(row['name'])

mc_shootings.add_to(map)

# Create a feature group for the school locations
school_locations = folium.FeatureGroup(name='School Locations')

# Style the school markers and circles
for index, row in boston_schools.iterrows():
    marker = folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=row['name'],
        icon=folium.Icon(color='blue', icon='home', prefix='fa')
    ).add_to(school_locations)
        
    folium.Circle(
        radius=200,
        location=[row['latitude'], row['longitude']],
        color="#3186cc",
        fill=True,
        fill_color="#3186cc",
        fill_opacity=0.4,
    ).add_to(school_locations)

# Add the school locations to the map
school_locations.add_to(map)

# Customize map appearance
folium.LayerControl().add_to(map)

# Create a label widget to display the number of crimes and crimes within the radius of schools
crimes_label = widgets.Label(value='Number of Crimes: 0')
crimes_within_radius_label = widgets.Label(value='Crimes within Radius: 0')
schools_within_radius_label = widgets.Label(value='Schools where Crimes Occurred within Radius:')

# Function to update the number of crimes and crimes within the radius based on the selected crime from the dropdown
def update_crimes_label(change):
    selected_crime = change['new']
    num_crimes = len(crimes_2021_2022_filtered[crimes_2021_2022_filtered['OFFENSE_DESCRIPTION'] == selected_crime])
    crimes_label.value = f'Number of Crimes: {num_crimes}'
    
    num_crimes_within_radius = crimes_within_radius.count(selected_crime)
    crimes_within_radius_label.value = f'Crimes within Radius: {num_crimes_within_radius}'
    
    if selected_crime in schools_within_radius:
        schools = schools_within_radius[selected_crime]
        schools_list = "\n".join(f"<li>{school.strip()}</li>" for school in schools)
        schools_within_radius_label.value = f'Schools where Crimes Occurred within Radius:<br><ul>{schools_list}</ul>'
    else:
        schools_within_radius_label.value = 'Schools where Crimes Occurred within Radius:'

# Create a dropdown widget with all different crimes
crime_dropdown = widgets.Dropdown(
    options=crimes_2021_2022_filtered['OFFENSE_DESCRIPTION'].unique(),
    description='Crime:'
)

# Create an HTML widget to display the schools within radius
schools_within_radius_label = widgets.HTML(value='Schools where Crimes Occurred within Radius:')

# Observe the change in the dropdown value and update the number of crimes and crimes within the radius
crime_dropdown.observe(update_crimes_label, 'value')

# Display the map and widgets
display(map)
display(widgets.VBox([crime_dropdown, crimes_label, crimes_within_radius_label, schools_within_radius_label]))


In [29]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import folium
from folium.plugins import HeatMap, MarkerCluster
import ipywidgets as widgets
from IPython.display import display

# --- Load Your Data (ensure you have these files) ---
# It's assumed you have already loaded 'crimes_2022_2023_filtered' and 'boston_schools' pandas DataFrames.
# crimes_2022_2023_filtered = pd.read_csv('your_crimes.csv')
# boston_schools = pd.read_csv('your_schools.csv')

# --- Convert to GeoDataFrames for Geospatial Operations ---
# Create a GeoDataFrame for crimes
crimes_gdf = gpd.GeoDataFrame(
    crimes_2022_2023_filtered,
    geometry=gpd.points_from_xy(crimes_2022_2023_filtered.Long, crimes_2022_2023_filtered.Lat),
    crs="EPSG:4326"  # Set Coordinate Reference System to WGS84
)

# Create a GeoDataFrame for schools
schools_gdf = gpd.GeoDataFrame(
    boston_schools,
    geometry=gpd.points_from_xy(boston_schools.longitude, boston_schools.latitude),
    crs="EPSG:4326"
)

# Convert OCCURRED_ON_DATE to datetime objects for temporal analysis
crimes_gdf['OCCURRED_ON_DATE'] = pd.to_datetime(crimes_gdf['OCCURRED_ON_DATE'])

In [30]:
# --- Perform Spatial Join for Proximity Analysis ---
# Project to a coordinate system that uses meters for accurate buffering (e.g., Web Mercator)
schools_projected = schools_gdf.to_crs("EPSG:3857")
crimes_projected = crimes_gdf.to_crs("EPSG:3857")

# Create a 200-meter buffer around each school
schools_projected['geometry'] = schools_projected.geometry.buffer(200)

# Perform the spatial join to find crimes within the school buffers
# 'op=within' ensures we only get crimes that are strictly inside a buffer
crimes_near_schools = gpd.sjoin(crimes_projected, schools_projected, how="inner", predicate='within')

# Convert back to the original CRS for mapping with folium
crimes_near_schools = crimes_near_schools.to_crs("EPSG:4326")

print(f"Found {len(crimes_near_schools)} crimes within 200 meters of a school.")

Found 44 crimes within 200 meters of a school.


In [31]:
# --- Create the Interactive Folium Map ---
# Center the map on Boston
map_center = [42.3601, -71.0589]
crime_map = folium.Map(location=map_center, zoom_start=12, tiles="CartoDB positron")

# --- Layer 1: Heatmap for Crime Density ---
# Create a list of lat/lon points for the heatmap
heat_data = [[point.xy[1][0], point.xy[0][0]] for point in crimes_gdf.geometry]
HeatMap(heat_data, radius=15).add_to(folium.FeatureGroup(name='Crime Density Heatmap').add_to(crime_map))

# --- Layer 2: Crime Incident Markers ---
mc_shootings = MarkerCluster(name='Crime Incidents')
for idx, row in crimes_gdf.iterrows():
    popup_content = f"<b>Offense:</b> {row['OFFENSE_DESCRIPTION']}<br><b>Date:</b> {row['OCCURRED_ON_DATE'].strftime('%Y-%m-%d %H:%M')}"
    marker = folium.Marker(
        location=[row.geometry.y, row.geometry.x],
        popup=popup_content,
        icon=folium.Icon(color='red', icon='exclamation-circle', prefix='fa')
    )
    mc_shootings.add_child(marker)
mc_shootings.add_to(crime_map)

# --- Layer 3: School Locations and Buffers ---
school_locations = folium.FeatureGroup(name='School Locations')
for idx, row in schools_gdf.iterrows():
    # School marker
    folium.Marker(
        location=[row.geometry.y, row.geometry.x],
        popup=row['name'],
        icon=folium.Icon(color='blue', icon='school', prefix='fa')
    ).add_to(school_locations)
    # School buffer zone circle
    folium.Circle(
        radius=200,
        location=[row.geometry.y, row.geometry.x],
        color="#1E90FF",
        fill=True,
        fill_opacity=0.3
    ).add_to(school_locations)
school_locations.add_to(crime_map)


# --- Add Layer Control to the map ---
folium.LayerControl().add_to(crime_map)

In [32]:
# --- Create Interactive Widgets ---
crime_dropdown = widgets.Dropdown(
    options=['All Crimes'] + sorted(crimes_gdf['OFFENSE_DESCRIPTION'].unique().tolist()),
    value='All Crimes',
    description='Filter by Crime:',
    style={'description_width': 'initial'}
)

total_crimes_label = widgets.Label()
crimes_near_school_label = widgets.Label()
schools_affected_label = widgets.HTML() # Use HTML widget for better formatting

# --- Function to Update Labels based on Dropdown ---
def update_dashboard(selected_crime):
    if selected_crime == 'All Crimes':
        filtered_total = crimes_gdf
        filtered_near_schools = crimes_near_schools
    else:
        filtered_total = crimes_gdf[crimes_gdf['OFFENSE_DESCRIPTION'] == selected_crime]
        filtered_near_schools = crimes_near_schools[crimes_near_schools['OFFENSE_DESCRIPTION'] == selected_crime]

    # Update the labels with new statistics
    total_crimes_label.value = f"Total Selected Crimes: {len(filtered_total):,}"
    crimes_near_school_label.value = f"Crimes Near Schools: {len(filtered_near_schools):,}"
    
    # Create a unique, sorted list of affected school names
    affected_schools = sorted(filtered_near_schools['name'].unique().tolist())
    
    if affected_schools:
        school_list_html = "".join(f"<li>{school}</li>" for school in affected_schools)
        schools_affected_label.value = f"<b>Affected Schools:</b><ul>{school_list_html}</ul>"
    else:
        schools_affected_label.value = "<b>Affected Schools:</b> None"

# Link the function to the dropdown's value
def on_dropdown_change(change):
    update_dashboard(change.new)

crime_dropdown.observe(on_dropdown_change, names='value')

# --- Display the Map and Widgets ---
# Initial call to populate the dashboard
update_dashboard('All Crimes')

# Arrange widgets vertically
dashboard = widgets.VBox([
    crime_dropdown,
    total_crimes_label,
    crimes_near_school_label,
    schools_affected_label
])

# Display everything
display(dashboard)
display(crime_map)

In [37]:
import pandas as pd
import geopandas as gpd
import folium
from folium.plugins import HeatMap, MarkerCluster
import ipywidgets as widgets
from IPython.display import display, HTML
import json
import requests
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from shapely.geometry import Polygon

# --- Configuration & Styling ---
# Use a professional and clean color palette
CHOROPLETH_PALETTE = 'YlOrRd'
SCHOOL_BUFFER_COLOR = "#1E90FF"
INCIDENT_COLOR = 'red'
SCHOOL_COLOR = 'blue'

# --- 1. Data Loading and Preprocessing ---
# It's assumed you have already loaded 'crimes_2022_2023_filtered' and 'boston_schools' pandas DataFrames.
# For reproducibility, here's how you might load them:
# crimes_df = pd.read_csv('your_crimes.csv')
# schools_df = pd.read_csv('your_schools.csv')
# For this example, we'll create placeholder DataFrames if they don't exist.
try:
    crimes_df = crimes_2022_2023_filtered.copy()
    schools_df = boston_schools.copy()
except NameError:
    print("Creating placeholder crime and school data...")
    # This is example data. Replace with your actual data loading.
    crimes_data = {'Lat': [42.35, 42.36, 42.34], 'Long': [-71.06, -71.07, -71.05],
                   'OFFENSE_DESCRIPTION': ['ASSAULT', 'LARCENY', 'ASSAULT'],
                   'OCCURRED_ON_DATE': ['2023-05-10 14:00', '2023-05-11 18:30', '2023-05-12 22:00']}
    schools_data = {'latitude': [42.355], 'longitude': [-71.065], 'name': ['Example School']}
    crimes_df = pd.DataFrame(crimes_data)
    schools_df = pd.DataFrame(schools_data)


# --- Fetch Neighborhood Boundaries from Boston Open Data ---
try:
    url = "https://boston.opendatasoft.com/explore/dataset/boston-neighborhoods/download/?format=geojson"
    neighborhoods_gdf = gpd.read_file(url)
    neighborhoods_gdf = neighborhoods_gdf.rename(columns={'Name': 'neighborhood_name'})
    print("Successfully loaded neighborhood boundaries.")
except Exception as e:
    print(f"Could not fetch neighborhood boundaries. Using a placeholder. Error: {e}")
    # Create a placeholder if the URL fails - a simple square polygon over downtown Boston
    placeholder_polygon = Polygon([(-71.07, 42.36), (-71.05, 42.36), (-71.05, 42.35), (-71.07, 42.35)])
    neighborhoods_gdf = gpd.GeoDataFrame(
        {'neighborhood_name': ['Downtown Placeholder']},
        geometry=[placeholder_polygon],
        crs="EPSG:4326"
    )


# --- Fetch Socioeconomic Data (Example: Population) ---
# In a real scenario, you would load a CSV with population/income per neighborhood.
# For this example, we'll create a sample DataFrame.
population_data = {
    'neighborhood_name': neighborhoods_gdf['neighborhood_name'].tolist(),
    'population': np.random.randint(5000, 50000, size=len(neighborhoods_gdf))
}
pop_df = pd.DataFrame(population_data)
neighborhoods_gdf = neighborhoods_gdf.merge(pop_df, on='neighborhood_name', how='left')
neighborhoods_gdf['population'] = neighborhoods_gdf['population'].fillna(1) # Avoid division by zero


# --- Convert DataFrames to GeoDataFrames ---
crimes_gdf = gpd.GeoDataFrame(
    crimes_df,
    geometry=gpd.points_from_xy(crimes_df.Long, crimes_df.Lat),
    crs="EPSG:4326"
)
schools_gdf = gpd.GeoDataFrame(
    schools_df,
    geometry=gpd.points_from_xy(schools_df.longitude, schools_df.latitude),
    crs="EPSG:4326"
)
crimes_gdf['OCCURRED_ON_DATE'] = pd.to_datetime(crimes_gdf['OCCURRED_ON_DATE'])


# --- 2. Spatial and Temporal Analysis ---

# --- Associate Crimes with Neighborhoods ---
crimes_by_neighborhood = gpd.sjoin(crimes_gdf, neighborhoods_gdf[['neighborhood_name', 'geometry']], how='inner', predicate='within')
# Drop the 'index_right' column immediately after the join to prevent conflicts later
crimes_by_neighborhood = crimes_by_neighborhood.drop(columns=['index_right'])
neighborhood_crime_counts = crimes_by_neighborhood['neighborhood_name'].value_counts().reset_index()
neighborhood_crime_counts.columns = ['neighborhood_name', 'crime_count']

# --- Calculate Crime Rate ---
neighborhoods_gdf = neighborhoods_gdf.merge(neighborhood_crime_counts, on='neighborhood_name', how='left')
neighborhoods_gdf['crime_count'] = neighborhoods_gdf['crime_count'].fillna(0)
# Calculate crime rate per 1,000 people
neighborhoods_gdf['crime_rate_per_1000'] = (neighborhoods_gdf['crime_count'] / neighborhoods_gdf['population']) * 1000


# --- Temporal Analysis: Day of Week and Hour ---
crimes_gdf['hour'] = crimes_gdf['OCCURRED_ON_DATE'].dt.hour
crimes_gdf['day_of_week'] = crimes_gdf['OCCURRED_ON_DATE'].dt.day_name()
temporal_heatmap_data = crimes_gdf.pivot_table(index='day_of_week', columns='hour', aggfunc='size', fill_value=0)
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
temporal_heatmap_data = temporal_heatmap_data.reindex(day_order)

# --- Generate Temporal Heatmap Image ---
fig, ax = plt.subplots(figsize=(12, 5))
sns.heatmap(temporal_heatmap_data, cmap='viridis', ax=ax, linewidths=.5)
ax.set_title('Crime Frequency by Hour and Day of the Week', fontsize=14)
ax.set_xlabel('Hour of Day')
ax.set_ylabel('Day of Week')
plt.tight_layout()
heatmap_path = 'temporal_heatmap.png'
plt.savefig(heatmap_path)
plt.close(fig)


# --- 3. Create the Interactive Map ---
map_center = [42.3601, -71.0589]
crime_map = folium.Map(location=map_center, zoom_start=12, tiles="CartoDB positron")

# --- Layer 1: Neighborhood Crime Rate Choropleth ---
choropleth = folium.Choropleth(
    geo_data=neighborhoods_gdf.to_json(),
    name='Neighborhood Crime Rate',
    data=neighborhoods_gdf,
    columns=['neighborhood_name', 'crime_rate_per_1000'],
    key_on='feature.properties.neighborhood_name',
    fill_color=CHOROPLETH_PALETTE,
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Crimes per 1,000 Residents',
    highlight=True
).add_to(crime_map)

# Add tooltips to the choropleth layer
folium.GeoJsonTooltip(
    fields=['neighborhood_name', 'crime_count', 'population', 'crime_rate_per_1000'],
    aliases=['Neighborhood:', 'Total Crimes:', 'Population:', 'Rate (per 1k):'],
    style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;")
).add_to(choropleth.geojson)


# --- Layer 2: Crime Density Heatmap ---
heat_data = [[point.xy[1][0], point.xy[0][0]] for point in crimes_gdf.geometry]
HeatMap(heat_data, radius=15, name='Crime Density').add_to(crime_map)

# --- Layer 3: School Locations and Buffers ---
school_locations = folium.FeatureGroup(name='School Locations & Buffers')
for _, row in schools_gdf.iterrows():
    folium.Marker(
        location=[row.geometry.y, row.geometry.x],
        popup=f"<b>{row['name']}</b>",
        icon=folium.Icon(color=SCHOOL_COLOR, icon='school', prefix='fa')
    ).add_to(school_locations)
    folium.Circle(
        radius=200,
        location=[row.geometry.y, row.geometry.x],
        color=SCHOOL_BUFFER_COLOR,
        fill=True,
        fill_opacity=0.2
    ).add_to(school_locations)
school_locations.add_to(crime_map)

# Add Layer Control
folium.LayerControl().add_to(crime_map)


# --- 4. Interactive Dashboard Widgets ---
crime_dropdown = widgets.Dropdown(
    options=['All Crimes'] + sorted(crimes_gdf['OFFENSE_DESCRIPTION'].unique().tolist()),
    value='All Crimes',
    description='Filter by Crime:',
    style={'description_width': 'initial'},
    layout={'width': '400px'}
)

# Use HTML widgets for richer text formatting
stats_output = widgets.HTML(layout={'border': '1px solid #ccc', 'padding': '10px', 'margin_top': '10px'})

# --- Function to Update Dashboard ---
def update_dashboard(selected_crime):
    if selected_crime == 'All Crimes':
        filtered_crimes = crimes_by_neighborhood
    else:
        filtered_crimes = crimes_by_neighborhood[crimes_by_neighborhood['OFFENSE_DESCRIPTION'] == selected_crime]

    total_crimes = len(filtered_crimes)
    
    # Proximity Analysis
    crimes_proj = gpd.GeoDataFrame(filtered_crimes, geometry=filtered_crimes.geometry).set_crs("EPSG:4326").to_crs("EPSG:3857")
    schools_proj = schools_gdf.to_crs("EPSG:3857")
    schools_proj['geometry'] = schools_proj.geometry.buffer(200)
    crimes_near_schools = gpd.sjoin(crimes_proj, schools_proj, how="inner", predicate='within')
    
    num_near_schools = len(crimes_near_schools)
    percent_near_schools = (num_near_schools / total_crimes * 100) if total_crimes > 0 else 0
    
    # Find the neighborhood with the highest count for the selected crime
    if not filtered_crimes.empty:
        top_neighborhood = filtered_crimes['neighborhood_name'].mode()[0]
    else:
        top_neighborhood = "N/A"

    stats_output.value = f"""
    <h3>Analysis for: {selected_crime}</h3>
    <p><b>Total Incidents:</b> {total_crimes:,}</p>
    <p><b>Incidents within 200m of a School:</b> {num_near_schools:,} ({percent_near_schools:.1f}%)</p>
    <p><b>Top Neighborhood for this Crime:</b> {top_neighborhood}</p>
    """

# Link function to dropdown
def on_dropdown_change(change):
    update_dashboard(change.new)

crime_dropdown.observe(on_dropdown_change, names='value')

# --- Display Everything ---
# Initial call to populate dashboard
update_dashboard('All Crimes')

# Display the temporal heatmap image
display(HTML(f'<h2>Temporal Crime Patterns</h2><img src="{heatmap_path}" alt="Temporal Heatmap">'))

# Display the interactive dashboard and map
display(HTML("<h2>Interactive Crime Map & Statistics</h2>"))
dashboard = widgets.VBox([crime_dropdown, stats_output])
display(dashboard)
display(crime_map)


Could not fetch neighborhood boundaries. Using a placeholder. Error: HTTP Error 404: Not Found
